In [1]:
from transformers import AutoConfig, AutoModelForCausalLM, AutoTokenizer
from CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM
from transformers import PreTrainedTokenizerFast
from transformers import XLMRobertaTokenizer

In [2]:
# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)

#tokenizer = PreTrainedTokenizerFast.from_pretrained("./CrispyTokenizer")

# XLM-Roberta tokenizer yükleniyor
#tokenizer = XLMRobertaTokenizer.from_pretrained("xlm-roberta-base")


# load the tokenizer and the model
tokenizer = AutoTokenizer.from_pretrained("../Tokenizer/Crispy Tokenizer/")


crispy_config = CrispyLLMConfig(attn_implementation="flash_attention_2", decoder_dropout=0.1 , attention_dropout=0.0, use_flash_attention_2=True, max_position_embeddings=4096, vocab_size=len(tokenizer.get_vocab()), n_heads=30, max_seq_len=4096, hidden_size=64*30, num_hidden_layers=30, dtype="bfloat16")

crispy_config._attn_implementation_autoset = True  # 👈 Buraya ekliyorsun

model = AutoModelForCausalLM.from_config(crispy_config)
    # 4. Kayıt işlemi
model.save_pretrained("../CrispyTest/CrispyLLM-RoPE")
tokenizer.save_pretrained("../CrispyTest/CrispyLLM-RoPE") 

('../CrispyTest/CrispyLLM-RoPE/tokenizer_config.json',
 '../CrispyTest/CrispyLLM-RoPE/special_tokens_map.json',
 '../CrispyTest/CrispyLLM-RoPE/tokenizer.json')

In [3]:
model

CrispyForCausalLM(
  (embedding): EmbeddingLayer(
    (token_embedding): TokenEmbedding(
      (embedding_layer): Embedding(50000, 1920)
    )
  )
  (decoderBlocks): ModuleList(
    (0-29): 30 x DecoderBlock(
      (attention_block): AttentionBlock(
        (qkv_proj): Linear(in_features=1920, out_features=5760, bias=True)
        (o_proj): Linear(in_features=1920, out_features=1920, bias=True)
        (rms_norm1): RMSNormBlock(
          (rmsNorm): RMSNorm((1920,), eps=1e-05, elementwise_affine=True)
        )
        (attn): FlashAttentionBlockBase()
        (rope): RotaryPositionalEmbedding()
      )
      (feedforward_network): FeedforwardNetwork(
        (ln1): Linear(in_features=1920, out_features=7680, bias=True)
        (swiglu): SwiGLU(
          (linear1): Linear(in_features=7680, out_features=3840, bias=True)
          (linear2): Linear(in_features=1920, out_features=7680, bias=True)
        )
        (ln2): Linear(in_features=7680, out_features=1920, bias=True)
      )
    

In [4]:
import torch

text = "Selam"

# 1️⃣ Tokenizer → Tensor
inputs = tokenizer(text, return_tensors="pt")

# 2️⃣ Modelin cihazına gönder (örneğin CUDA)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
inputs = {k: v.to(device) for k, v in inputs.items()}
model = model.to(device)

# 3️⃣ Inference
model.eval()
with torch.no_grad():
    outputs = model(**inputs)

logits = outputs.logits
print("✅ Logits shape:", logits.shape)


✅ Logits shape: torch.Size([1, 4, 50000])


In [5]:
""" "auto_map": {
    "AutoConfig": "modeling_crispy.CrispyLLMConfig",
    "AutoModelForCausalLM": "modeling_crispy.CrispyForCausalLM"
  }, """

' "auto_map": {\n    "AutoConfig": "modeling_crispy.CrispyLLMConfig",\n    "AutoModelForCausalLM": "modeling_crispy.CrispyForCausalLM"\n  }, '

In [6]:
import os
os.getcwd()

'/media/hosman/Yerel Disk D/Codes/Basic LLM Train/MyLLM'

In [7]:
""" import torch
from transformers import AutoModelForCausalLM
from transformers import AutoConfig, AutoModelForCausalLM
from CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM
from transformers import PreTrainedTokenizerFast

# 3. Kayıt (Auto ile kullanabilmek için)
AutoConfig.register("crispy", CrispyLLMConfig)
AutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)

model = AutoModelForCausalLM.from_pretrained(
    "./CrispyLLM_RoPE2",
    #use_flash_attention_2=True,
    attn_implementation="flash_attention_2",  # veya "eager"
    trust_remote_code=True,
    torch_dtype=torch.bfloat16,
    device_map="auto"
)

#tokenizer = PreTrainedTokenizerFast.from_pretrained("./CrispyLLM-RoPE")

model """

' import torch\nfrom transformers import AutoModelForCausalLM\nfrom transformers import AutoConfig, AutoModelForCausalLM\nfrom CrispyLLM_RoPE2.modeling_crispy_rope import CrispyLLMConfig, CrispyForCausalLM\nfrom transformers import PreTrainedTokenizerFast\n\n# 3. Kayıt (Auto ile kullanabilmek için)\nAutoConfig.register("crispy", CrispyLLMConfig)\nAutoModelForCausalLM.register(CrispyLLMConfig, CrispyForCausalLM)\n\nmodel = AutoModelForCausalLM.from_pretrained(\n    "./CrispyLLM_RoPE2",\n    #use_flash_attention_2=True,\n    attn_implementation="flash_attention_2",  # veya "eager"\n    trust_remote_code=True,\n    torch_dtype=torch.bfloat16,\n    device_map="auto"\n)\n\n#tokenizer = PreTrainedTokenizerFast.from_pretrained("./CrispyLLM-RoPE")\n\nmodel '

In [8]:
# Hugging Face'e gönder
model.push_to_hub("hosmankarabulut/Crispy-2.8B-CLM")         # kendi kullanıcı adınla
tokenizer.push_to_hub("hosmankarabulut/Crispy-2.8B-CLM")

model-00002-of-00002.safetensors:   0%|          | 0.00/726M [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Upload 2 LFS files:   0%|          | 0/2 [00:00<?, ?it/s]

README.md:   0%|          | 0.00/5.17k [00:00<?, ?B/s]

CommitInfo(commit_url='https://huggingface.co/hosmankarabulut/Crispy-2.8B-CLM/commit/0ef7772b31f47b4c23d4dcbb0346813e2ac9cd92', commit_message='Upload tokenizer', commit_description='', oid='0ef7772b31f47b4c23d4dcbb0346813e2ac9cd92', pr_url=None, repo_url=RepoUrl('https://huggingface.co/hosmankarabulut/Crispy-2.8B-CLM', endpoint='https://huggingface.co', repo_type='model', repo_id='hosmankarabulut/Crispy-2.8B-CLM'), pr_revision=None, pr_num=None)